# Class 14

* Continuing on k-fold cross validation in python
* Helps ensure your validation scores are more robust and meaningful, by training on different sets of data by sub-sampling across your taining set.

**So... how do we do this in scikit-learn**


In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import TimeSeriesSplit


## How to think about causal impact in GradientBoosting

In [5]:
# first, let's fit our model, using our standard setup
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
# read in data
df = pd.read_csv('../data/restaurants.csv')
df.drop(['calendar_date', 'visit_date'], axis=1, inplace=True)
# fill missing values
df = df.fillna(0)
# declare X & y
X = df.drop('visitors', axis=1)
y = df['visitors']
# make pipeline
pipe = make_pipeline(ce.TargetEncoder(), GradientBoostingRegressor())
# fit
pipe.fit(X, y)
# and score
pipe.score(X, y)

/usr/local/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.47122281492324647

**Scoring Regressions** 

Lots of ways you can score a regression. R-squared tends to be the most common.

### Calculation of R-squared

R-squared = 1 - sum[(yi - yhati) ** 2]/sum[(yi - ymean) ** 2]

In [6]:
pipe.predict(X) # This is y-hat

array([20.94833128, 20.16374384, 25.05294918, ..., 52.35427745,
       45.47308493, 51.68725966])

In [7]:
y - pipe.predict(X) #This is our error column, y - y-hat

0          4.051669
1         11.836256
2          3.947051
3         -5.507854
4        -11.834932
            ...    
252103     2.888309
252104     7.889550
252105    16.645723
252106   -14.473085
252107   -25.687260
Name: visitors, Length: 252108, dtype: float64

In [8]:
(y - pipe.predict(X))**2 # Squared errors, to keep it positive

0          16.416019
1         140.096960
2          15.579210
3          30.336453
4         140.065609
             ...    
252103      8.342326
252104     62.244992
252105    277.080079
252106    209.470187
252107    659.835309
Name: visitors, Length: 252108, dtype: float64

In [9]:
np.sum((y - pipe.predict(X))**2) # Sum of squared errors

37432646.37172356

In [17]:
our_model_error = np.sum((y - pipe.predict(X))**2)

In [11]:
y.mean()

20.973761245180636

In [12]:
naive_model_error = np.sum((y-y.mean())**2)

In [18]:
rsq = 1 - (our_model_error / naive_model_error)
print(f"R-squared is {rsq}")

R-squared is 0.47122281492324647


### What does r-squared imply?
***
* If R-squared is zero, then you are effectively just predicting the mean from the data directly; and X drivers are not helping you move away from the mean.
***
* Negative R-squared typically means something has gone seriously wrong, as you are creating errors larger than the mean squared error, from your data. Since most models start using the mean as a base you should almost always see some sort of improvement.
***
* R-squared has no units. So consistent across circumstance.

## Talking about feature importance of GBM

In [19]:
# let's create our feature importance dataframe
feats = pd.DataFrame({
    'Columns': X.columns,
    'Importance': pipe[1].feature_importances_
}).sort_values(by='Importance', ascending=False)
# and here we go
feats

,Columns,Importance
0,id,0.875754
1,day_of_week,0.105299
2,holiday,0.006767
7,reserve_visitors,0.004578
6,longitude,0.004431
5,latitude,0.002049
4,area,0.000582
3,genre,0.000540


## So what is feature importance? How do we figure it out?

* Feature importance score was developed so we could understand how different variables are contributing.

In [20]:
feats['Importance'].sum()

0.9999999999999999



### **Permutation Feature Importance**
* How much will your model score change if you randomly shuffle values of a particular column value?
***
* By randomly shuffling the values within the column, we effectively turn that column into 'noise'
* Then we scale them back to 1.
* This let's us see how much a single feature drives an outcome
***
* Can be used as a cutoff to remove data / columns from the model
* Usually better to focus on explanatory power rather than fixed number of columns
***
*But it does have weaknesses*
* It is only a positive number, so it doesn't tell you the direction of the impact.
* It doesn't tell you **how sensitive** your target variable is to any particular factor in X

### Boosting....
Tends to be good a picking up slight statistical data and doesn't really worry about having too many columns (unlike say regression).

If a column isn't very important, it is unlikely to be included in a specific boosing tree in any particular round of tree building. So won't impact the model much at all.

Tops of trees tend to be fairly reliable, and only run into trouble as you go deeper.

In [24]:
# Take a copy of X
X_copy = X.copy()

# Shuffle the id column
X_copy['id'] = X_copy['id'].sample(frac=1).values

In [25]:
X['id']

0         air_ba937bf13d40fb24
1         air_ba937bf13d40fb24
2         air_ba937bf13d40fb24
3         air_ba937bf13d40fb24
4         air_ba937bf13d40fb24
                  ...         
252103    air_a17f0778617c76e2
252104    air_a17f0778617c76e2
252105    air_a17f0778617c76e2
252106    air_a17f0778617c76e2
252107    air_a17f0778617c76e2
Name: id, Length: 252108, dtype: object

In [26]:
X_copy['id']

0         air_fa4ffc9057812fa2
1         air_add9a575623726c8
2         air_97159fc4e90053fe
3         air_915558a55c2bc56c
4         air_694571ea13fb9e0e
                  ...         
252103    air_63a88d81295195ed
252104    air_28064154614b2e6c
252105    air_4de6d887a7b1c1fc
252106    air_f267dd70a6a6b5d3
252107    air_42c9aa6d617c5057
Name: id, Length: 252108, dtype: object

In [27]:
# Score the column based on the random 
pipe.score(X_copy,y)

-0.36806842560720665

In [28]:
# Calculate the difference vs the actual data.
np.abs(pipe.score(X_copy,y) - pipe.score(X,y))

0.8392912405304531

In [29]:
# So to understand how it links to r-squared; do this for each column, 
# then rescore with the shuffled data


# let's do this for all of our columns
cols       = []
impact     = []
for column in X.columns:
    X_copy         = X.copy()
    X_copy[column] = X_copy[column].sample(frac=1).values
    total_impact   = np.abs(pipe.score(X_copy, y) - pipe.score(X, y))
    cols.append(column)
    impact.append(total_impact)
# and turn it into a dataframe
feats = pd.DataFrame({'Column': cols,
                      'Impact': impact}).sort_values(by='Impact', ascending=False)

In [30]:
feats 

,Column,Impact
0,id,0.835523
1,day_of_week,0.100809
2,holiday,0.007491
6,longitude,0.004609
7,reserve_visitors,0.004105
5,latitude,0.001377
4,area,0.000418
3,genre,0.000402


In [33]:
# Then scale back to R-squared 
feats['Impact']/feats['Impact'].sum()

0    0.875137
1    0.105589
2    0.007846
6    0.004827
7    0.004300
5    0.001442
4    0.000438
3    0.000421
Name: Impact, dtype: float64

In [34]:
feats2 = feats['Impact']/feats['Impact'].sum()

In [39]:
feats2.cumsum()< 0.99

0     True
1     True
2     True
6    False
7    False
5    False
4    False
3    False
Name: Impact, dtype: bool

## Looking at Partial Dependence within Gradient Boosting
* Similar idea to understanding marginal impact / incremental change of target variable compared to the underlying variable
* To do this with a non-linear model we look at:
    * Partial Dependence
    * Individual Conditional Dependence

In [42]:
# Trees that are 5 deep
# 500 trees
# gives a lot of model parameters
(2**5)*500

16000

Basic approach: 
1. Fit the model first
2. Take a single column, choose a value, and change every single value to that one, generate predictions
3. Take the same column, choose a different value, change every single value to that one, generate predictions
4. Look at the difference in the predictions

In [46]:
# make a copy of X, to simplify things
X_copy = X.copy()

# Create an empty dataframe
preds = pd.DataFrame()

# we'll change every value in the copy to Monday
X_copy['day_of_week'] = 'Monday'
preds['Monday'] = pipe.predict(X_copy)
# Then Tuesday
X_copy['day_of_week'] = 'Tuesday'
preds['Tuesday'] = pipe.predict(X_copy)

In [44]:
X_copy.head()

,id,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors
0,air_ba937bf13d40fb24,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0
1,air_ba937bf13d40fb24,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0
2,air_ba937bf13d40fb24,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0
3,air_ba937bf13d40fb24,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0
4,air_ba937bf13d40fb24,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0


In [47]:
preds
# Now - for every single row, we have the predicted value for Monday,
# and then the predicted value for Tuesday.

,Monday,Tuesday
0,17.834932,18.915649
1,17.834932,18.915649
2,17.834932,18.915649
3,17.834932,18.915649
4,17.834932,18.915649
...,...,...
252103,37.035296,38.591633
252104,37.666372,39.837884
252105,37.621192,39.792704
252106,45.473085,46.051193


In [48]:
preds.mean() # Average impact on Monday vs Tuesday

Monday     16.687075
Tuesday    17.938631
dtype: float64

In [49]:
preds.mean().diff() # Average change of Tuesday vs Monday

Monday          NaN
Tuesday    1.251556
dtype: float64

In [50]:
# Loop to do every day in a full week:

# we'll do a loop and derive the same values for each unique day of the week
days_of_week = df['day_of_week'].unique()
# make a copy of X -- makes it easier
X_copy = X.copy()
# an empty dataframe
preds  = pd.DataFrame()
# loop through each unique value in the day_of_week column
for day in days_of_week:
    # set the value for the entire column during that day
    X_copy['day_of_week'] = day
    # look at our new predicted values with the adjusted column
    preds[day] = pipe.predict(X_copy)

In [51]:
preds

,Wednesday,Thursday,Friday,Saturday,Monday,Tuesday,Sunday
0,20.948331,20.163744,25.052949,27.507854,17.834932,18.915649,24.671304
1,20.948331,20.163744,25.052949,27.507854,17.834932,18.915649,24.671304
2,20.948331,20.163744,25.052949,27.507854,17.834932,18.915649,24.671304
3,20.948331,20.163744,25.052949,27.507854,17.834932,18.915649,24.671304
4,20.948331,20.163744,25.052949,27.507854,17.834932,18.915649,24.671304
...,...,...,...,...,...,...,...
252103,40.867392,40.262585,46.111691,50.812908,37.035296,38.591633,51.376021
252104,42.290367,41.685560,47.534667,52.110450,37.666372,39.837884,52.673564
252105,42.245186,41.640379,47.489486,51.791164,37.621192,39.792704,52.354277
252106,44.888994,44.941017,49.723604,51.307395,45.473085,46.051193,52.818498


In [52]:
preds.mean()

Wednesday    19.353997
Thursday     18.936975
Friday       23.169965
Saturday     26.149694
Monday       16.687075
Tuesday      17.938631
Sunday       23.805693
dtype: float64

In [57]:
preds = preds[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]

In [58]:
preds.mean()

Monday       16.687075
Tuesday      17.938631
Wednesday    19.353997
Thursday     18.936975
Friday       23.169965
Saturday     26.149694
Sunday       23.805693
dtype: float64

In [60]:
preds.mean().diff().cumsum()
# Impact of the predicted change from one day to another

Monday            NaN
Tuesday      1.251556
Wednesday    2.666923
Thursday     2.249901
Friday       6.482890
Saturday     9.462619
Sunday       7.118618
dtype: float64

In [62]:
pd.options.plotting.backend = "plotly"

preds.mean().diff().fillna(0).plot(title="Expected Impact for Different Days of the Week on Attendance vs prior day")

In [63]:
# !pip install pdpbox
# Cancelled and installed via terminal

Start setting keys..
Keys set.
     |████████████████████████████████| 57.7 MB 13.2 MB/s eta 0:00:01     |███████████████████████▊        | 42.8 MB 13.2 MB/s eta 0:00:02
     |████████████████████████████████| 10.3 MB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 29.1 MB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 11.1 MB/s eta 0:00:01
  Using cached joblib-0.17.0-py3-none-any.whl (301 kB)
     |████████████████████████████████| 465 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 12.0 MB/s eta 0:00:01
  Installing build dependencies ... |^C
anceled
ERROR: Operation cancelled by user
